In [20]:
import cv2
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
info = pd.read_csv("_info.csv")
info

,FileID,Negative,Typical,Atypical
0,0003b2210c64,0,1,0
1,00af6f8c2a3d,1,0,0
2,00c9033fbc2e,0,0,1
3,00e0ce73dac8,0,0,1
4,00f0a591f18a,0,1,0
...,...,...,...,...
1195,ff60a32aa049,0,0,1
1196,ff9666e69d19,0,1,0
1197,ff971972490d,1,0,0
1198,ffb804652089,1,0,0


In [30]:
y_OneHot = np.array(info[['Negative','Typical','Atypical']])
y = np.argmax(y_OneHot, axis=1)

In [17]:
fileids = info['FileID'].tolist()
X = []
for fileid in fileids:
    pixel_array = plt.imread("img/train/{}.jpg".format(fileid))
    pixel_array = cv2.resize(pixel_array, (256,256))   # some size
    pixel_array = pixel_array.reshape(-1)
    X.append(pixel_array)
    
X = np.array(X)
X = X.astype("float16")
# 檢查
print(X.shape)

(1200, 65536)


# train、test dataset

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# random forest

In [38]:
clf = RandomForestClassifier(max_depth=500, max_features=500)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

# save
joblib.dump(clf, "model/rf_20211116.joblib")

0.49166666666666664


# Xgboost

In [42]:
from sklearn import metrics
from xgboost.sklearn import XGBClassifier


clf = XGBClassifier(
        #樹的個數
        n_estimators=1000,
        # 如同學習率
        learning_rate= 0.3, 
        # 構建樹的深度，越大越容易過擬合    
        max_depth=50, 
        # 隨機取樣訓練樣本 訓練例項的子取樣比
        subsample=1, 
        # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        gamma=0, 
        # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        reg_lambda=1,  
        
        #最大增量步長，我們允許每個樹的權重估計。
        max_delta_step=0,
        # 生成樹時進行的列取樣 
        colsample_bytree=1, 

        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=1, 

        #隨機種子
        seed=1000 
        
        # L1 正則項引數
#        reg_alpha=0,
        
        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #scale_pos_weight=1,
        
        #多分類的問題 指定學習任務和相應的學習目標
        #objective= 'multi:softmax', 
        
        # 類別數，多分類與 multisoftmax 並用
        #num_class=10,
        
        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        silent=0 ,
        # cpu 執行緒數 預設最大
#        nthread=4,
    
        #eval_metric= 'auc'
)

# 模型 訓練
clf.fit(X_train,y_train,eval_metric='auc')
# 預測值
y_pred=clf.predict(X_test)
# 真實值 賦值
y_true= y_test

# 計算精度
print("Accuracy : %.2g" % metrics.accuracy_score(y_true, y_pred))

# save
joblib.dump(clf, "model/xgboost_20211116.joblib")

/home/frank100/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy : 0.5


['model/xgboost_20211116.joblib']